In [11]:
import pathlib, json, csv

with open("/scr/ig_pipeline/artifacts/pipeline/collision_average_volumes_2.json", "r") as f:
    d = json.load(f)
    collision_average_volumes = d["average_volumes"]
    collision_volumes = d["volumes"]

# For now, get categories from CSV file
categories_by_id = {}
avg_category_specs = {}
with open("/scr/ig_pipeline/metadata/category_mapping.csv", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        # TODO: Use something more authoritative than this entry
        # Get the has_system entry to check if the category is a particle system,
        # and if so, skip it entirely.
        has_system = row["has_system"].strip().lower()
        if has_system == "true":
            continue

        cat_id = i  # Temporarily just use row idx. TODO: Cover everything
        category = row["category"].strip()
        categories_by_id[cat_id] = category

        assert category in collision_average_volumes, f"Category {category} not in collision_average_volumes"

        volume = collision_average_volumes[category]
        mass = float(row["mass (auto)"]) if row["mass (auto)"] and row["mass (auto)"] != "#DIV/0!" else None
        assert mass is not None and mass > 0, f"Invalid mass for category {category}"
        density = mass / volume if mass and volume else None

        avg_category_specs[category] = {
            "mass": mass,
            "volume": volume,
            "density": density,
        }

In [12]:
avg_category_specs

{'ceilings': {'mass': 2000.0,
  'volume': 2.049506745528556,
  'density': 975.844555946661},
 'walls': {'mass': 2000.0,
  'volume': 0.8257256371381512,
  'density': 2422.1120309788594},
 'floors': {'mass': 2000.0,
  'volume': 4.202978695726735,
  'density': 475.8529949327238},
 'armchair': {'mass': 20.0,
  'volume': 0.08338315221811032,
  'density': 239.85660733579365},
 'bathtub': {'mass': 110.0,
  'volume': 0.24395564798785854,
  'density': 450.90163276512703},
 'bed': {'mass': 50.0,
  'volume': 1.3161169053882489,
  'density': 37.990546124966166},
 'bench': {'mass': 25.0,
  'volume': 0.17590564975337367,
  'density': 142.1216432505206},
 'bottom_cabinet': {'mass': 50.0,
  'volume': 0.022965389239366214,
  'density': 2177.1893120928385},
 'bottom_cabinet_no_top': {'mass': 40.0,
  'volume': 0.045932306362725196,
  'density': 870.84675618337},
 'breakfast_table': {'mass': 18.0,
  'volume': 0.07168814910614167,
  'density': 251.08752596400765},
 'burner': {'mass': 0.5,
  'volume': 0.011

In [13]:
from bddl.knowledge_base import *

In [14]:
# Load particle params and delete everything that shows up there.
import csv
synset_has_particle_density = set()
with open("/scr/ig_pipeline/metadata/substance_hyperparams.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        synset = Synset.get(row["synset"].strip())
        if synset is None:
            print(f"Synset {row['synset']} not found")
            continue
        has_density = "particle_density" in json.loads(row["hyperparams"])
        if has_density:
            synset_has_particle_density.add(synset)
        elif "visualSubstance" not in synset.property_names:
            print(f"Synset {synset} does not have particle density")

synset_is_visual = {s for s in Synset.all_objects() if "visualSubstance" in s.property_names}
ignore_synsets = synset_has_particle_density | synset_is_visual
ignore_categories = {c for s in ignore_synsets for c in s.categories}

In [15]:
densities = {Category.get(k): v["density"] for k, v in avg_category_specs.items() if Category.get(k) not in ignore_categories}
keys, values = zip(*sorted(densities.items(), key=lambda x: -x[1]))

In [16]:
# Print the first 100 values
for i in range(100):
    print(f"{keys[i]}: {values[i]}")

swimming_pool: 470033.07858111535
moth_ball: 203660.98188455973
thumbtack: 93608.01477102708
ruby: 82730.09691471994
tweezers: 81760.20547299243
bulldog_clip: 59788.08567838683
straight_pin: 59273.02468250953
clam: 54764.94832808579
ring: 47245.48319084842
half_peppermint: 43271.903203002745
half_white_turnip: 42404.32583319093
key: 39889.15273467888
bicycle_chain: 32808.03694046838
staple: 29474.128033034016
shears: 28888.70301093055
white_turnip: 28739.939776873776
gold_coin: 28722.651829469476
bracelet: 25723.76494931834
paper_clip: 24659.506866330667
silver_coin: 21983.219965038057
half_broccoli: 19789.944299205326
diamond: 19451.057199134157
clothesline_rope: 19255.761122575837
half_radish: 17766.27924269636
half_bacon: 17501.43309886315
toothpick: 17214.897631438704
paving_stone: 16498.039661811425
wall_nail: 15976.972620394989
garlic_clove: 15892.374272685915
jade_roller: 15805.605744853134
flower_petal: 15125.971184389651
clout_nail: 15104.603667439012
half_agave: 14184.6818437

In [17]:
# Print the last 100 values
for i in range(100):
    print(f"{keys[-i-1]}: {values[-i-1]}")

background: 0.32819101555109814
rail_fence: 1.7357722973472982
balloon: 2.0074663283762266
volleyball_net: 2.2698055905520556
tablecloth: 2.4007242346881723
tights: 2.8590186558073696
curtain: 3.4160715699115185
dress_shirt: 3.568244511671636
diaper: 4.948165265683657
sauna_suit: 5.116693613860933
tent: 5.4447541112507345
garden_plant: 5.470855671017875
plastic_bag: 6.793874825950612
stairs: 7.024924066526396
garden_umbrella: 7.6474163005604066
hoodie: 7.73684735710165
vine: 7.96935533710391
information_bulletin: 8.522986909229921
pillowcase: 8.714073810251678
receipt: 9.138613317022575
pants: 9.663626567548304
legging: 9.843955481750625
t_shirt: 10.909273396511027
wading_pool: 11.276489814825121
dress: 11.530906528685044
sleeping_bag: 11.554795724314053
sweatshirt: 11.616793435674879
onesie: 11.810543599035842
christmas_tree_decorated: 12.184241452846765
skirt: 12.858996517423266
pottable_beefsteak_tomato: 13.588589798194509
chicken_wire: 14.008576352914982
ceremonial_robe: 14.2740887

In [18]:
# How many categories are heavier than 2,000 kg/m^3? (e.g. concrete)
heavy_categories = [c for c, d in densities.items() if d > 2000]
print(f"Number of categories heavier than 2,000 kg/m^3: {len(heavy_categories)}")

# How many categories are lighter than 1 kg/m^3? (e.g. air)
light_categories = [c for c, d in densities.items() if d < 1]
print(f"Number of categories lighter than 1 kg/m^3: {len(light_categories)}")

Number of categories heavier than 2,000 kg/m^3: 289
Number of categories lighter than 1 kg/m^3: 1


In [25]:
# Heaviest and lightest objects in the dataset
masses = {}
for cat_name, mdls in collision_volumes.items():
    if cat_name in ["floors", "ceilings", "walls", "driveway", "lawn", "background", "roof"]:
        continue
    cat = Category.get(cat_name)
    if cat in ignore_categories:
        continue
    if cat not in densities:
        print(f"Category {cat} not in densities")
        continue
    for mdl, volume in mdls.items():
        mass = densities[cat] * volume
        masses[f"{cat}-{mdl}"] = mass
sorted_masses = sorted(masses.items(), key=lambda x: -x[1])

# Print the heaviest and lightest objects
print("Heaviest objects:")
for i in range(1000):
    print(f"{sorted_masses[i][0]}: {sorted_masses[i][1]}")


Category peanut not in densities
Category fennel not in densities
Category charcoal not in densities
Heaviest objects:
swimming_pool-vnvmkx: 77793484.29816838
swimming_pool-kohria: 75000.0
low_resolution_tree-elrvnt: 70886.87950090502
pillar-xznmxo: 36516.40941612613
pillar-zjotnf: 36002.47363887693
pillar-keesuv: 25578.550750137176
swimming_pool-qjhauf: 23933.159794532618
bush-frqici: 18220.42761289397
skeletal_frame-cvtaos: 15839.798709322073
lorry-oxbgfh: 15000.0
fixed_window-kmvdjo: 12136.317783325614
trailer_truck-dnvcsu: 12000.0
skeletal_frame-cfqmqw: 10234.895882729585
paver-sekqqq: 9275.54113930008
downlight-tghdrr: 9184.360835314403
pillar-oukzkz: 7963.741980210607
stone_wall-shqfvd: 6000.0
stadium_stand-psxwbh: 5979.827237149793
boulder-axxuoc: 5658.211392673271
boulder-vxbkjq: 5207.327136362595
garden_statue-smqgfh: 5000.0
pillar-afgytu: 4788.114020754348
bush-kjakgd: 4217.67673689942
stadium_stand-mtoaja: 4020.172762850208
bush-oedeqi: 3383.4037650001346
pillar-vzzejz: 3336